##### TODO:
- Separate array values
- Add functionality to the utils module

In [1]:
import numpy as np
import pandas as pd

from utils.utils import get_data

## Default Data Cleanup

#### Load a single dataframe

In [2]:
EE_default_015 = get_data("data/EE_015/", ["default"])
EE_default_050 = get_data("data/EE_050/", ["default"])
EE_default_069 = get_data("data/EE_069/", ["default"])
EE_default = pd.concat([EE_default_015, EE_default_050, EE_default_069], ignore_index=True, axis=0)

#### Drop ID

In [17]:
EE_default.drop("ID", axis=1, inplace=True)

#### Split FILTER into separate columns (omit PASS and FAIL)

https://docs.varsome.com/en/variant-calling-and-quality-filters

In [3]:
unique_filters = set(';'.join(EE_default['FILTER']).split(';'))
unique_filters = [f for f in unique_filters if f != "PASS" and f != "FAIL"]
unique_filters

['haplotype',
 'strand_bias',
 'multiallelic',
 'base_qual',
 'germline',
 'fragment',
 'map_qual',
 'weak_evidence',
 'clustered_events',
 'slippage']

#### Create new columns names

In [4]:
filter_cols = ["FILTER_" + f for f in unique_filters]
filter_cols

['FILTER_haplotype',
 'FILTER_strand_bias',
 'FILTER_multiallelic',
 'FILTER_base_qual',
 'FILTER_germline',
 'FILTER_fragment',
 'FILTER_map_qual',
 'FILTER_weak_evidence',
 'FILTER_clustered_events',
 'FILTER_slippage']

In [5]:
for f_col, f_val in zip(filter_cols, unique_filters):
    EE_default[f_col] = EE_default['FILTER'].apply(lambda x: 1 if f_val in x else 0)

EE_default_clean = EE_default.drop("FILTER", axis=1)
EE_default_clean.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER_haplotype,FILTER_strand_bias,FILTER_multiallelic,FILTER_base_qual,FILTER_germline,FILTER_fragment,FILTER_map_qual,FILTER_weak_evidence,FILTER_clustered_events,FILTER_slippage
0,chr1,15820,rs2691315,G,T,.,0,0,0,0,0,0,0,0,0,0
1,chr1,17385,rs201535981,G,A,.,0,0,0,0,0,0,0,0,0,0
2,chr1,17697,rs71260069,G,C,.,0,0,0,0,0,0,0,0,0,0
3,chr1,133129,rs367730352,G,A,.,0,0,0,0,0,0,0,0,0,0
4,chr1,183629,rs71267774,G,A,.,0,0,0,0,0,0,0,0,0,0


In [10]:
EE_default_clean[filter_cols].sum(axis=0)

FILTER_haplotype           14501
FILTER_strand_bias          1032
FILTER_multiallelic         3614
FILTER_base_qual             562
FILTER_germline             1075
FILTER_fragment              315
FILTER_map_qual            10247
FILTER_weak_evidence        1423
FILTER_clustered_events    20008
FILTER_slippage             6497
dtype: int64

## CSQ Data Cleanup

#### Load a single dataframe

In [11]:
EE_csq_015 = get_data("data/EE_015/", ["csq"])
EE_csq_050 = get_data("data/EE_050/", ["csq"])
EE_csq_069 = get_data("data/EE_069/", ["csq"])
EE_csq = pd.concat([EE_csq_015, EE_csq_050, EE_csq_069], ignore_index=True, axis=0)

In [12]:
EE_csq.columns

Index(['Allele', 'Consequence', 'IMPACT', 'SYMBOL', 'Gene', 'Feature_type',
       'Feature', 'BIOTYPE', 'EXON', 'INTRON', 'HGVSc', 'HGVSp',
       'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids',
       'Codons', 'Existing_variation', 'DISTANCE', 'STRAND', 'FLAGS',
       'SYMBOL_SOURCE', 'HGNC_ID', 'CANONICAL', 'MANE_SELECT',
       'MANE_PLUS_CLINICAL', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'SWISSPROT',
       'TREMBL', 'UNIPARC', 'UNIPROT_ISOFORM', 'REFSEQ_MATCH', 'SOURCE',
       'REFSEQ_OFFSET', 'GIVEN_REF', 'USED_REF', 'BAM_EDIT', 'SIFT',
       'PolyPhen', 'DOMAINS', 'HGVS_OFFSET', 'AF', 'AFR_AF', 'AMR_AF',
       'EAS_AF', 'EUR_AF', 'SAS_AF', 'gnomADe_AF', 'gnomADe_AFR_AF',
       'gnomADe_AMR_AF', 'gnomADe_ASJ_AF', 'gnomADe_EAS_AF', 'gnomADe_FIN_AF',
       'gnomADe_NFE_AF', 'gnomADe_OTH_AF', 'gnomADe_SAS_AF', 'gnomADg_AF',
       'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 'gnomADg_ASJ_AF',
       'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF', 

In [13]:
EE_csq.shape

(444444, 96)

#### Drop uninteresting columns

In [14]:
drop_columns = ["TSL", "APPRIS", "CCDS", "ENSP", "SWISSPROT", "TREMBL", "UNIPARC", "UNIPROT_ISOFORM", "REFSEQ_MATCH",
                "SOURCE", "REFSEQ_OFFSET", "GIVEN_REF", "USED_REF", "BAM_EDIT", "DOMAINS", "HGVS_OFFSET", "AF", "AFR_AF",
                "AMR_AF", "EAS_AF", "EUR_AF", "SAS_AF"]
drop_columns_gnomad = [c for c in EE_csq if c.startswith("gnom") and c != "gnomADe_AF" and c!= "gnomADg_AF"]
drop_columns.extend(drop_columns_gnomad)
EE_potential_csq = EE_csq.drop(drop_columns, axis=1)
EE_potential_csq.shape

(444444, 56)

#### Drop potentially interesting columns

In [15]:
potential_drop_columns = ["Consequence", "IMPACT", "CANONICAL", "MANE_SELECT", "MANE_PLUS_CLINICAL", "SIFT", "PolyPhen",
                          "CLIN_SIG", "EVE_CLASS", "EVE_SCORE", "CADD_PHRED", "CADD_RAW", "LOEUF", "NMD", "SpliceAI_pred_DP_AG",
                          "SpliceAI_pred_DP_AL", "SpliceAI_pred_DP_DG", "SpliceAI_pred_DP_DL", "SpliceAI_pred_DS_AG",
                          "SpliceAI_pred_DS_AL", "SpliceAI_pred_DS_DG", "SpliceAI_pred_DS_DL", "SpliceAI_pred_SYMBOL"]

EE_important_csq = EE_potential_csq.drop(potential_drop_columns, axis=1)
EE_important_csq.shape

(444444, 33)

#### Separate array values

In [40]:
separable_csq_columns = ["PHENOTYPES", "CLIN_SIG", "SOMATIC", "DOMAINS", "PUBMED", "TREMBL", "Consequence", "HGVSp",
                         "FLAGS", "PHENO", "Existing_variation", "SWISSPROT", "TRANSCRIPTION_FACTORS", "VAR_SYNONYMS"]
potentially_separable_columns = ["CDS_position", "cDNA_position", "HGNC_ID", "SIFT", "PolyPhen", "Protein_position"]
check_correctness = ["Codons"]

## Genotype Data Cleanup

#### Load EE_15 with EE_50 and EE_69 separately

In [18]:
EE_genotype_015 = get_data("data/EE_015/", ["genotype"])
EE_genotype_050 = get_data("data/EE_050/", ["genotype"])
EE_genotype_069 = get_data("data/EE_069/", ["genotype"])

#### Remove excess columns from EE_069

In [22]:
EE_genotype_069.shape

(140174, 46)

In [20]:
excess_columns = set(EE_genotype_069.columns).difference(set(EE_genotype_050.columns))
excess_columns

{'AC',
 'AF',
 'AN',
 'BaseQRankSum',
 'ClippingRankSum',
 'ExcessHet',
 'FS',
 'MLEAC',
 'MLEAF',
 'MQ',
 'MQRankSum',
 'QD',
 'ReadPosRankSum',
 'SOR'}

In [24]:
cut_EE_genotype_069 = EE_genotype_069.drop(excess_columns, axis=1)
cut_EE_genotype_069.shape

(140174, 32)

#### Delete uninteresting columns from EE_069

In [28]:
drop_columns_acmg =  [c for c in cut_EE_genotype_069.columns if c.startswith("ACMG") and c!="ACMG_class"]
drop_columns_gnomad = [c for c in cut_EE_genotype_069 if c.startswith("gnom") and c != "gnomadExomes_AF" and c!= "gnomadGenomes_AF"]

EE_potential_genotype_069 = cut_EE_genotype_069.drop(drop_columns_acmg, axis=1)
EE_potential_genotype_069 = EE_potential_genotype_069.drop(drop_columns_gnomad, axis=1)
EE_potential_genotype_069 = EE_potential_genotype_069.drop("DP", axis=1)
EE_potential_genotype_069.columns

Index(['ACMG_class', 'CGDinheritance', 'ClinVarClass', 'ClinVarDisease',
       'DANN_score', 'Gene', 'MutationTaster_pred', 'MutationTaster_score',
       'SIFT_score', 'coding_impact', 'function', 'gnomadExomes_AF',
       'gnomadGenomes_AF', 'hgvs'],
      dtype='object')

#### Delete potentially interesting columns from EE_069

In [29]:
potential_drop_columns = ["ClinVarClass", "ClinVarDisease", "DANN_score", "MutationTaster_pred", "MutationTaster_score", "SIFT_score"]

EE_important_genotype_069 = EE_potential_genotype_069.drop(potential_drop_columns, axis=1)
EE_important_genotype_069.columns

Index(['ACMG_class', 'CGDinheritance', 'Gene', 'coding_impact', 'function',
       'gnomadExomes_AF', 'gnomadGenomes_AF', 'hgvs'],
      dtype='object')

#### Combine EE_015 and EE_050

In [30]:
EE_genotype_015_050 = pd.concat([EE_genotype_015, EE_genotype_050], ignore_index=True, axis=0)

#### Delete uninteresting columns from EE_050 and EE_015

In [31]:
drop_columns_acmg_amp =  [c for c in EE_genotype_015_050.columns if (c.startswith("ACMG") or c.startswith("AMP")) and c!="ACMG_class"]
drop_columns_gnomad = [c for c in EE_genotype_015_050 if c.startswith("gnom") and c != "gnomadExomes_AF" and c!= "gnomadGenomes_AF"]

EE_potential_genotype_015_050 = EE_genotype_015_050.drop(drop_columns_acmg_amp, axis=1)
EE_potential_genotype_015_050 = EE_potential_genotype_015_050.drop(drop_columns_gnomad, axis=1)
EE_potential_genotype_015_050 = EE_potential_genotype_015_050.drop("DP", axis=1)
EE_potential_genotype_015_050 = EE_potential_genotype_015_050.drop("MMQ", axis=1)
EE_potential_genotype_015_050.columns

Index(['ACMG_class', 'AS_FilterStatus', 'AS_SB_TABLE', 'CGDinheritance',
       'ClinVarClass', 'ClinVarDisease', 'DANN_score', 'ECNT', 'GERMQ', 'Gene',
       'MBQ', 'MFRL', 'MPOS', 'MutationTaster_pred', 'MutationTaster_score',
       'POPAF', 'RPA', 'RU', 'SIFT_score', 'STR', 'STRQ', 'TLOD',
       'coding_impact', 'cosmicFathMMPrediction', 'cosmicFathMMScore',
       'function', 'gnomadExomes_AF', 'gnomadGenomes_AF', 'hgvs'],
      dtype='object')

#### Delete potentially interesting columns from EE_050 and EE_015

In [35]:
potential_drop_columns = ["ClinVarClass", "ClinVarDisease", "DANN_score", "MutationTaster_pred", "MutationTaster_score", "SIFT_score"]

EE_important_genotype_015_050 = EE_potential_genotype_015_050.drop(potential_drop_columns, axis=1)
EE_important_genotype_015_050.columns

Index(['ACMG_class', 'AS_FilterStatus', 'AS_SB_TABLE', 'CGDinheritance',
       'ECNT', 'GERMQ', 'Gene', 'MBQ', 'MFRL', 'MPOS', 'POPAF', 'RPA', 'RU',
       'STR', 'STRQ', 'TLOD', 'coding_impact', 'cosmicFathMMPrediction',
       'cosmicFathMMScore', 'function', 'gnomadExomes_AF', 'gnomadGenomes_AF',
       'hgvs'],
      dtype='object')

#### Propose two combinations of EE_015 EE_050 with EE_069

In [39]:
# Concatenate with all columns from EE_015 EE_050
EE_potential_genotype_all = pd.concat([EE_potential_genotype_015_050, EE_potential_genotype_069], ignore_index=True, axis=0)
EE_important_genotype_all = pd.concat([EE_important_genotype_015_050, EE_important_genotype_069], ignore_index=True, axis=0)

# Concatenate with only common columns
different_potential_columns = set(EE_potential_genotype_015_050.columns).difference(set(EE_potential_genotype_069.columns))
EE_important_genotype_common = pd.concat([EE_potential_genotype_015_050.drop(different_potential_columns, axis=1), EE_potential_genotype_069], ignore_index=True, axis=0)

different_important_columns = set(EE_important_genotype_015_050.columns).difference(set(EE_important_genotype_069.columns))
EE_important_genotype_common = pd.concat([EE_important_genotype_015_050.drop(different_potential_columns, axis=1), EE_important_genotype_069], ignore_index=True, axis=0)

#### Separate array values